In [1]:
import sqlite3
import pandas as pd

In [2]:
database = './data/PizzaRunner.sqlite'

In [3]:
connection = sqlite3.connect(database)

In [4]:
cursor = connection.cursor()

# RUNNERS Table

In [5]:
pd.read_sql("SELECT * FROM runners", connection)

,runner_id,registration_date
0,1,2021-01-01
1,2,2021-01-03
2,3,2021-01-08
3,4,2021-01-15
4,1,2021-01-01
5,2,2021-01-03
6,3,2021-01-08
7,4,2021-01-15


**No cleaning required**

# CUSTOMER ORDERS Table

In [6]:
pd.read_sql("SELECT * FROM customer_orders", connection)

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,None,None,2020-01-01 18:05:02
1,2,101,1,None,None,2020-01-01 19:00:52
2,3,102,1,None,None,2020-01-02 23:51:23
3,3,102,2,None,None,2020-01-02 23:51:23
4,4,103,1,4,None,2020-01-04 13:23:46
5,4,103,1,4,None,2020-01-04 13:23:46
6,4,103,2,4,None,2020-01-04 13:23:46
7,5,104,1,None,1,2020-01-08 21:00:29
8,6,101,2,None,None,2020-01-08 21:03:13
9,7,105,2,None,1,2020-01-08 21:20:29


#### Cleaning

In [7]:
update_customer_orders_table = """UPDATE customer_orders 
                                    SET exclusions = '', 
                                    extras = ''
                                    WHERE 
                                    (
                                    exclusions IS NULL
                                    OR exclusions LIKE 'null'
                                    OR extras IS NULL
                                    OR extras LIKE 'null'
                                    );"""

In [8]:
cursor.execute(update_customer_orders_table)

In [9]:
pd.read_sql("SELECT * FROM customer_orders", connection)

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,,,2020-01-01 18:05:02
1,2,101,1,,,2020-01-01 19:00:52
2,3,102,1,,,2020-01-02 23:51:23
3,3,102,2,,,2020-01-02 23:51:23
4,4,103,1,,,2020-01-04 13:23:46
5,4,103,1,,,2020-01-04 13:23:46
6,4,103,2,,,2020-01-04 13:23:46
7,5,104,1,,,2020-01-08 21:00:29
8,6,101,2,,,2020-01-08 21:03:13
9,7,105,2,,,2020-01-08 21:20:29


# RUNNER ORDERS Table

In [10]:
pd.read_sql("SELECT * FROM runner_orders", connection)

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,None
1,2,1,2020-01-01 19:10:54,20km,27 minutes,None
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,4,2,2020-01-04 13:53:03,23.4,40,None
4,5,3,2020-01-08 21:10:57,10,15,None
5,6,3,None,None,None,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,None
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,None
8,9,2,None,None,None,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,None


#### Cleaning

In [11]:
update_pizza_runners_table = """UPDATE runner_orders 
                                SET 
                                pickup_time =
                                (CASE
                                    WHEN pickup_time IS NULL OR pickup_time LIKE 'null' THEN ' '
                                    ELSE pickup_time
                                END),
                                distance = 
                                (CASE
                                    WHEN distance IS NULL OR distance LIKE 'null' THEN ' '
                                    WHEN distance LIKE '%km' THEN TRIM('km', distance)
                                    ELSE distance 
                                END),
                                duration = 
                                (CASE 
                                    WHEN duration IS NULL OR duration LIKE 'null' THEN ' '
                                    WHEN duration LIKE '%mins' THEN TRIM('mins', duration)
                                    WHEN duration LIKE '%minute' THEN TRIM('minute', duration)
                                    WHEN duration LIKE '%minutes' THEN TRIM('minutes', duration)
                                    ELSE duration
                                  END),
                                  cancellation = 
                                  (CASE
                                      WHEN cancellation IS NULL OR cancellation LIKE 'null' THEN ' '
                                      ELSE cancellation
                                   END)
                                    ;"""

In [12]:
cursor.execute(update_pizza_runners_table)

In [13]:
pd.read_sql("SELECT * FROM runner_orders", connection)

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,,,
1,2,1,2020-01-01 19:10:54,,,
2,3,1,2020-01-03 00:12:37,,,
3,4,2,2020-01-04 13:53:03,23.4,40,
4,5,3,2020-01-08 21:10:57,10,15,
5,6,3,,,,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,,,
7,8,2,2020-01-10 00:15:02,,,
8,9,2,,,,Customer Cancellation
9,10,1,2020-01-11 18:50:20,,,


# PIZZA NAMES Table

In [14]:
pd.read_sql("SELECT * FROM pizza_names", connection)

,pizza_id,pizza_name
0,1,Meatlovers
1,2,Vegetarian


**No cleaning required**

# PIZZA RECIPES Table

In [15]:
pd.read_sql("SELECT * FROM pizza_recipes", connection)

,pizza_id,toppings
0,1,"1, 2, 3, 4, 5, 6, 8, 10"
1,2,"4, 6, 7, 9, 11, 12"


**No cleaning required**

# PIZZA TOPPINGS Table

In [16]:
pd.read_sql("SELECT * FROM pizza_toppings", connection)

,topping_id,topping_name
0,1,Bacon
1,2,BBQ Sauce
2,3,Beef
3,4,Cheese
4,5,Chicken
5,6,Mushrooms
6,7,Onions
7,8,Pepperoni
8,9,Peppers
9,10,Salami


**No cleaning required**